<img style="float: left;" src="./images/PallenceAI-Final.png" width="800">

# Sentiment Analysis on IMDb Reviews

### Sequence Models: Transformers

<img style="float: left;" src="./images/sequence-transformer.png" width="300">

<img style="float: left;" src="./images/imdb2.png" width="400">

### Import needed libraries 

In [1]:
#Basic Python packages for data wrangling
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

from pathlib import Path

#Tensorflow & Keras related packages
import tensorflow as tf
from tensorflow import keras
from keras import layers

from utils import plot_history

### Load IMDb Dataset Preloaded in Keras

In [2]:
import tensorflow_datasets as tfds

train, val, test = tfds.load(
    name="imdb_reviews",
    split=["train[:80%]", "train[80%:]", "test"],
    as_supervised=True)

C:\Users\sridh\anaconda3\envs\tf2.10_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Understanding the Data 

In [3]:
# Lets see how many sample reviews are there in each dataset
print (len(train))
print (len(val))
print (len(test))

20000
5000
25000


In [4]:
# Batch the data
train_data = train.batch(32)
val_data = val.batch(32)
test_data =test.batch(32)

In [5]:
train_data

<BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [6]:
#lets look at the first batch of data

for reviews, labels in train_data.take(1):
    print ("Reviews shape", reviews.shape)
    print ("Labels shape", labels.shape, "\n")
    
    print ('First Review:', reviews[0].numpy().decode("utf-8"), "\n") 
    print ('First Label:', labels[0].numpy())

Reviews shape (32,)
Labels shape (32,) 

First Review: This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it. 

First Label: 0


In [7]:
# First select only reviews from training data (which contains both reviews/labels) 
# ..which we will vectorize into numeric format.

train_data_onlyreviews = train_data.map(lambda x,y : x) # given a tuple (x,y), output only x

<img style="float: left;" src="./images/vectorization.png" width="300">

## Data prep for Sequence Models
1. Vectorize the sequences as shown below with TextVectorization in Keras:
* Tokenize the sequences
* Encode the tokens into integers

2. After this we can convert each integer to a corresponding vector (such as one-hot, or word embeddings). This can be 
done by a seperate one-hot layer or embedding layer directly during model building stage

**Vectorize by encoding to integers first**

In [8]:
# Keras has inbuilt Text Vectorization layer that can standardize, tokenize, and 
# ..convert to indices or token vectors

Text Vectorization in Keras by default
* converts to lower case
* removes punctuation
* split on whitepsace

In [9]:
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization

# this is a limit on sequence length. each sequence will be 300 words long
max_length = 300 

# this is for the vocabulary limit. max tokens to use for creating the vocabulary. 
max_tokens = 10000 

text_vectorization = TextVectorization(
    max_tokens=max_tokens,
    output_mode = "int",
    output_sequence_length = max_length
)

In [10]:
# Apply vectorization on training data reviews

text_vectorization.adapt(train_data_onlyreviews) 

In [11]:
# Each review is standardized, tokenized, and convered into an integer sequence

train_int = train_data.map(lambda x,y: (text_vectorization(x),y)) 
val_int = val_data.map(lambda x,y: (text_vectorization(x),y)) 
test_int = test_data.map(lambda x,y: (text_vectorization(x),y)) 

In [12]:
# lets look again at the first batch of data, now that reviews are converted into integers

for reviews, labels in train_int.take(1):
    print ("Reviews shape", reviews.shape)
    print ("Labels shape", labels.shape, "\n")
    print ('First Review:', reviews[0], "\n")
    print ('First Label:', labels[0].numpy())

# Each review is converted into a 300 length integer sequence. Each batch has 32 reviews

Reviews shape (32, 300)
Labels shape (32,) 

First Review: tf.Tensor(
[  11   14   34  411  376   18   90   27    1    8   33 1319 4152   41
  503    1  193   25   86  152   19   11  216  316   27   65  241  214
    8  485   56   65   86  115   95   22 5749   11   93  635  738   11
   18    7   34  398 9897  169 2480  410    2   88 1203  137   67  144
   52    2    1 7096   67  245   65 2937   16    1 2793    1    1 1444
 5053    3   40    1 1663   17 4152   14  157   19    4 1203  853 7809
    8    4   18   12   14 3837    5   98  146 1221   10  231  688   12
   48   25   93   39   11 7514  152   39 1319    1   50  409   10   95
 1156  845  140    9    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0  

## 1. Build the Transformer Encoder Architecture

In [13]:
# Reset the seeds for model initialization/training for reproducibility

random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [20]:
# Parameters

max_tokens = 10000  # Size of the vocabulary
max_length = 300  # Maximum length of a sequence

embed_dim = 128  # Embedding dimension
num_heads = 4  # Number of attention heads
ff_dim = 128  # Dimension of the feed-forward network within the transformer

<img style="float: left;" src="./images/transformer_encoder2.jpg" width="800">

**Transformer Encoder Block**

In [21]:
# Define the Transformer Encoder block using built-in layers

def transformer_encoder(inputs, embed_dim, num_heads, ff_dim):
    
    # Multi-Head Attention
    attn_output = layers.MultiHeadAttention(key_dim=embed_dim, num_heads=num_heads)(inputs, inputs, inputs)
    
    # Residual Connection: # adding attn_output & inputs
    residual_connection1 = inputs + attn_output
    
    # Layer Normalization
    out1 = layers.LayerNormalization(epsilon=1e-6)(residual_connection1) 
    
    # Feed-Forward Network
    ffn_output = layers.Dense(ff_dim, activation="relu")(out1)
    ffn_output = layers.Dense(embed_dim)(ffn_output)
   
    # Residual Connection: Adding ffn_output & out1
    residual_connection2 = out1 + ffn_output
    
    # Layer Normalization
    out2 = layers.LayerNormalization(epsilon=1e-6)(residual_connection2) 
    
    return out2

**Word Order with Fixed Positional Embeddings using sine/cosine functions**

In [22]:
# Define Positional Encodings

def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(max_length, d_model):
    angle_rads = get_angles(np.arange(max_length)[:, np.newaxis], 
                            np.arange(d_model)[np.newaxis, :], d_model)
    
    # Apply the sin function to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    
    # Apply the cos function to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    
    # from (position, d_model)) to (batch_size, position, d_model) for broadcasting
    pos_encoding = angle_rads[np.newaxis, ...] 
    
    return tf.cast(pos_encoding, dtype=tf.float32)

**Build the complete Model**

In [23]:
# Build the model

def model_transformer_encoder(): 
    
    # Define Input shape
    inputs = keras.Input(shape = (max_length,), dtype="int64") 
    
    # Define word embedding layer.
    # Word embeddings of each token are calculated as part of training.
    embeddings = layers.Embedding(input_dim = max_tokens, output_dim=embed_dim, mask_zero=True)(inputs) 
      
    
    # Positional embeddings are fixed here. 
    pos_encoding = positional_encoding(max_length, embed_dim)
    
    # Add word embeddings & fixed positional embeddings
    embeddings = embeddings + pos_encoding
    
    # Adding Transformer Encoder layers
    x = transformer_encoder(embeddings, embed_dim, num_heads, ff_dim)
    # this layer ultimately results in Context aware embeddings after training
    
    # GlobalMaxPooling1D selects most important feature from each wordembedding across entire sentence 
    # Effectively summarizing the essence of sentence into a single vector 
    x = layers.GlobalMaxPooling1D()(x)
    
    #  Dropout layer
    x = layers.Dropout(0.5)(x)
    
    # Final output layer
    outputs = layers.Dense(1, activation="sigmoid")(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [24]:
model = model_transformer_encoder()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 300)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 300, 128)     1280000     ['input_2[0][0]']                
                                                                                                  
 tf.__operators__.add (TFOpLamb  (None, 300, 128)    0           ['embedding[0][0]']              
 da)                                                                                              
                                                                                                  
 multi_head_attention (MultiHea  (None, 300, 128)    263808      ['tf.__operators__.add[0][0]'

In [25]:
model = model_transformer_encoder()
path = Path("./models/model_transformer_encoder_fixedpositions.keras")

In [26]:
# We will reuse this function to train and evaluate for convenience
def train_evaluate(model,path,train,val,test):
    
    # call backs
    checkpoint_cb = keras.callbacks.ModelCheckpoint(filepath = path,
                                                       save_best_only=True) # Save only best model
    
    earlystop_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, 
                                                 restore_best_weights=True)
    callbacks = [checkpoint_cb,earlystop_cb]

    # Compile the model
    model.compile(optimizer="rmsprop", loss='binary_crossentropy',  metrics = ["accuracy"])
    
    # Train the model
    history = model.fit(train, validation_data = val, callbacks=callbacks, epochs=20)
    
    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(test)
    
    return (history,test_accuracy)

In [27]:
(history_te,test_accuracy_te) = train_evaluate(model,path,
                                               train_int,
                                               val_int,
                                               test_int)

Epoch 1/20
625/625 [==============================] - 28s 35ms/step - loss: 0.7345 - accuracy: 0.4985 - val_loss: 0.6957 - val_accuracy: 0.4938
Epoch 2/20
625/625 [==============================] - 18s 29ms/step - loss: 0.6469 - accuracy: 0.5964 - val_loss: 0.5553 - val_accuracy: 0.7292
Epoch 3/20
625/625 [==============================] - 17s 28ms/step - loss: 0.4849 - accuracy: 0.7741 - val_loss: 0.4092 - val_accuracy: 0.8204
Epoch 4/20
625/625 [==============================] - 19s 30ms/step - loss: 0.3522 - accuracy: 0.8490 - val_loss: 0.3452 - val_accuracy: 0.8474
Epoch 5/20
625/625 [==============================] - 19s 31ms/step - loss: 0.2728 - accuracy: 0.8899 - val_loss: 0.3437 - val_accuracy: 0.8530
Epoch 6/20
625/625 [==============================] - 18s 29ms/step - loss: 0.2196 - accuracy: 0.9148 - val_loss: 0.3724 - val_accuracy: 0.8464
Epoch 7/20
625/625 [==============================] - 19s 31ms/step - loss: 0.1846 - accuracy: 0.9302 - val_loss: 0.3654 - val_accuracy:

In [28]:
print (f"Accuracy on the test data set is {test_accuracy_te}")

Accuracy on the test data set is 0.847000002861023


## 2. Transformer Encoder with Learnable Positional Embeddings

In [34]:
# For this we will define custom classes subclassing the keras layer class

In [35]:
# Reset the seeds for model initialization/training for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [36]:
# Parameters
max_tokens = 10000  # Size of the vocabulary
max_length = 300  # Maximum length of a sequence

embed_dim = 128  # Embedding dimension
num_heads = 4  # Number of attention heads
ff_dim = 128  # Dimension of the feed-forward network within the transformer

### Build Positional Embedding Layer custom class

**What Do Learnable Positional Embeddings Learn?**

* In essence, learnable positional embeddings give the model the freedom to learn how position interacts with the token embeddings and the task at hand. 
* While we know the positions explicitly (1, 2, 3, etc.), the model might learn that certain positions (e.g., the middle or end of a sequence) should be weighted more heavily for certain tasks.

For example:

* For sentiment analysis, words near the end of a sentence might carry more weight because they often summarize the overall sentiment.

In [37]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, max_tokens, embed_dim, **kwargs):
        super().__init__(**kwargs)
        
        # Word embedding layer
        self.token_embeddings = layers.Embedding(input_dim=max_tokens, output_dim=embed_dim, 
                                                 mask_zero=True)
        # Position embeddings
        self.position_embeddings = layers.Embedding(input_dim=sequence_length, output_dim=embed_dim) 
        
        # both the above embeddings are initialized randomly first and 
        # ....will be calculated as part of training process.
        
        self.sequence_length = sequence_length
        self.max_tokens = max_tokens
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return self.token_embeddings.compute_mask(inputs, mask)
    
        #The compute_mask method in a custom layer ensures that the 
        #...masking information is correctly propagated through the layers.
        # without this the mask may not be propagated properly through the sebsequent layers.

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "sequence_length": self.sequence_length,
            "max_tokens": self.max_tokens,
        })
        return config

### Build Transformer Encoder Layer Custom Class

In [38]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        
        #Attention Layers
        self.attention = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        
        self.ffn = keras.Sequential([
            layers.Dense(ff_dim, activation="relu"),
            layers.Dense(embed_dim),])
        
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        

    def call(self, inputs, mask=None):
        if mask is not None:
            # reshape since the attention layer expects 3d or 4d: 
            # ..(batch_size, num_heads, seq_length, seq_length)
            mask = mask[:, tf.newaxis, :] 
        
        # Attention
        attn_output = self.attention(inputs, inputs, attention_mask=mask)  
        
        # Residual connection
        residual_connection1 = inputs + attn_output    
        
        # Layer Normalization
        out1 = self.layernorm1(residual_connection1)
        
        # Feed forward dense layer
        ffn_output = self.ffn(out1)
        
        # Residual connection
        residual_connection2 = out1 + ffn_output
        
        # Layer Normalization
        out2 = self.layernorm2(residual_connection2)
        
        return out2


    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "ff_dim": self.ff_dim,
        })
        return config

**Build the Model**

In [39]:
def model_transformer_encoder2():
    
    # Define input shape
    inputs = keras.Input(shape=(max_length,), dtype="int64") 

    # Positional Embedding Layer using the above defined position embedding class custom layer
    x = PositionalEmbedding(sequence_length=max_length, max_tokens=max_tokens, embed_dim=embed_dim)(inputs)
      
    # Transformer Encoder Layer
    x = TransformerEncoder(embed_dim=embed_dim, ff_dim=ff_dim, num_heads=num_heads)(x)
    
    # GlobalMaxPooling1D layer
    x = layers.GlobalMaxPooling1D()(x)
    
    # Dropout before the final classification layer
    x = layers.Dropout(0.5)(x) 
    
    # Final Dense layer
    outputs = layers.Dense(1, activation="sigmoid")(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [40]:
# Create the model
model = model_transformer_encoder2()
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 300)]             0         
                                                                 
 positional_embedding (Posit  (None, 300, 128)         1318400   
 ionalEmbedding)                                                 
                                                                 
 transformer_encoder (Transf  (None, 300, 128)         297344    
 ormerEncoder)                                                   
                                                                 
 global_max_pooling1d_2 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                           

In [41]:
# only difference between the previous model & the above one in terms of total parameters
# We have an additional 300 * 128 = 38400 paramters for positional embeddings 
#...which will now be learned instead of being fixed.

In [42]:
# 1577473+(300*128) = 1,615,873

In [43]:
model = model_transformer_encoder2()
path = Path("./models/model_transformer_encoder_learnedpositions.keras")

In [44]:
(history_te2,test_accuracy_te2) = train_evaluate(model,path,
                                                 train_int,
                                                 val_int,
                                                 test_int)

Epoch 1/20
625/625 [==============================] - 25s 34ms/step - loss: 0.4896 - accuracy: 0.7605 - val_loss: 0.2970 - val_accuracy: 0.8758
Epoch 2/20
625/625 [==============================] - 20s 32ms/step - loss: 0.2701 - accuracy: 0.8920 - val_loss: 0.2929 - val_accuracy: 0.8788
Epoch 3/20
625/625 [==============================] - 20s 31ms/step - loss: 0.2112 - accuracy: 0.9194 - val_loss: 0.2898 - val_accuracy: 0.8816
Epoch 4/20
625/625 [==============================] - 19s 31ms/step - loss: 0.1768 - accuracy: 0.9331 - val_loss: 0.3184 - val_accuracy: 0.8742
Epoch 5/20
625/625 [==============================] - 19s 30ms/step - loss: 0.1562 - accuracy: 0.9415 - val_loss: 0.3041 - val_accuracy: 0.8804
Epoch 6/20
625/625 [==============================] - 19s 31ms/step - loss: 0.1390 - accuracy: 0.9482 - val_loss: 0.3873 - val_accuracy: 0.8722
Epoch 7/20
625/625 [==============================] - 19s 31ms/step - loss: 0.1250 - accuracy: 0.9545 - val_loss: 0.3549 - val_accuracy:

In [45]:
print (f"Accuracy on the test data set is {test_accuracy_te2}")

Accuracy on the test data set is 0.8672800064086914
